Feature Engineering ใน Notebook นี้จัดทำขึ้นเพื่อ clean feature ทั้ง Numerical และ Categorical โดยมีทดลองจัดการกับ Value เหล่านี้ 2 แบบ
* แบบที่ 1 :
    * Categorical Feature : group ข้อมูลเป็นกลุ่มใหญ่ ๆ และตัด row ที่มี missing value ออก
    * Numerical : นำ Row ที่เป็น Outlier และ Missing Value ออก
* แบบที่ 2 :
    * Categorical Feature : group ข้อมูลเป็นกลุ่มใหญ่ ๆ และเติม Missing value ด้วย Mode
    * Numerical : เติม Missing value ด้วย median และ cap ค่า outlier

In [1]:
from src.clean_data import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

In [2]:
def remove_outliers(df, numerical_columns):
    """ลบ outlier ออกจากคอลัมน์ที่กำหนดโดยใช้ IQR"""
    df_clean = df.copy()  # สร้างสำเนา DataFrame เพื่อลบแถว
    
    for column in numerical_columns:
        Q1 = df_clean[column].quantile(0.25)  # ค่ามัธยฐานล่าง (25th percentile)
        Q3 = df_clean[column].quantile(0.75)  # ค่ามัธยฐานบน (75th percentile)
        IQR = Q3 - Q1  # ค่าความกว้างระหว่างควอไทล์
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # กรองแถวที่ไม่มี outlier
        df_clean = df_clean[(df_clean[column] >= lower_bound) & (df_clean[column] <= upper_bound)]
    
    return df_clean

In [3]:
def scale_data(data,scale_by='standard_scaler',target_col = 'มูลค่าความเสียหาย'):
    if scale_by == 'standard_scaler':
        # ใช้ StandardScaler
        scaler = StandardScaler()
    elif scale_by == 'min_max':
        scaler = MinMaxScaler()
        
    features = data.drop(target_col,axis=1)
    scaled_features = scaler.fit_transform(features)

    # สร้าง DataFrame ใหม่ที่สเกลแล้ว
    df_scaled = pd.DataFrame(scaled_features, columns=features.columns)

    # นำคอลัมน์ target กลับมา
    df_scaled[target_col] = data[target_col]
    return df_scaled

In [4]:
raw_data = pd.read_csv('./data/accident2024 + $Damage.csv')
target = 'มูลค่าความเสียหาย'
df = raw_data.copy()

# นำ Column ที่ไม่เกี่ยวข้องออก

# df = raw_data.drop(irrelevant_column,axis=1)


In [5]:
# ระบุ column ที่เป็น categorical และ numerical
categorical_feature = ['บริเวณที่เกิดเหตุ', 'มูลเหตุสันนิษฐาน',
               'ลักษณะการเกิดอุบัติเ', 'สภาพอากาศ']
numerical_feature = ['รถจักรยานยนต์', 'รถสามล้อเครื่อง',
       'รถยนต์นั่งส่วนบุคคล', 'รถตู้', 'รถปิคอัพโดยสาร', 'รถโดยสารมากกว่า4ล้อ',
       'รถปิคอัพบรรทุก4ล้อ', 'รถบรรทุก6ล้อ', 'รถบรรทุกไม่เกิน10ล้อ',
       'รถบรรทุกมากกว่า10ล้อ', 'รถอีแต๋น', 'อื่นๆ', 'คนเดินเท้า',
       'จำนวนผู้เสียชีวิต', 'จำนวนผู้บาดเจ็บสาหัส', 'จำนวนผู้บาดเจ็บเล็ก',
       'รวมจำนวนผู้บาดเจ็บ']

province_json_path = 'thailand_provinces.json'

### แบบที่ 1 :
* Categorical Feature : group ข้อมูลเป็นกลุ่มใหญ่ ๆ และตัด row ที่มี missing value ออก
* Numerical : นำ Row ที่เป็น Outlier และ Missing Value ออก

In [6]:
columns_to_clean = [
    'จังหวัด', 'รถคันที่1', 'บริเวณที่เกิดเหตุ', 'มูลเหตุสันนิษฐาน', 'ลักษณะการเกิดอุบัติเ', 
    'สภาพอากาศ', 'จำนวนรถที่เกิดเหตุ', 'จำนวนที่เกิดเหตุทั้ง', 'รถจักรยานยนต์', 'รถสามล้อเครื่อง', 
    'รถยนต์นั่งส่วนบุคคล', 'รถตู้', 'รถปิคอัพโดยสาร', 'รถโดยสารมากกว่า4ล้อ', 'รถปิคอัพบรรทุก4ล้อ', 
    'รถบรรทุก6ล้อ', 'รถบรรทุกไม่เกิน10ล้อ', 'รถบรรทุกมากกว่า10ล้อ', 'รถอีแต๋น', 'อื่นๆ', 'คนเดินเท้า', 
    'จำนวนผู้เสียชีวิต', 'จำนวนผู้บาดเจ็บสาหัส', 'จำนวนผู้บาดเจ็บเล็ก', 'รวมจำนวนผู้บาดเจ็บ', 'มูลค่าความเสียหาย'
]

In [7]:
cleaned_1_df = df.dropna(subset=columns_to_clean)
cleaned_1_df = remove_outliers(cleaned_1_df,numerical_feature)

cleaned_1_df = group_categorical_feature(cleaned_1_df)
cleaned_1_df = recalculate_accident_number(cleaned_1_df)

# Model Development
to_delete = ['LATITUDE', 'LONGITUDE','จังหวัด', 'รถคันที่1', 'บริเวณที่เกิดเหตุ', 'มูลเหตุสันนิษฐาน',
             'ลักษณะการเกิดอุบัติเ', 'สภาพอากาศ']

model_1 = cleaned_1_df.drop(to_delete,axis=1)
model_1 = encode_categorical_features(model_1,target_col = 'มูลค่าความเสียหาย')

# Scale Different Way
model_1_standard = scale_data(model_1,scale_by='standard_scaler')
model_1_minmax = scale_data(model_1,scale_by='min_max')

In [8]:
cleaned_1_df.to_csv('./output/data_after_clean_1.csv',encoding='utf-8-sig',index = False)

### แบบที่ 2 :
* Categorical Feature : group ข้อมูลเป็นกลุ่มใหญ่ ๆ และเติม Missing value ด้วย Mode
* Numerical : เติม Missing value ด้วย median และ cap ค่า outlier

In [8]:
cleaned_2_df = clean_by_cap_and_fill(df,numerical_feature,categorical_feature,province_json_path)

# Model Development
to_delete = ['LATITUDE', 'LONGITUDE','จังหวัด', 'รถคันที่1', 'บริเวณที่เกิดเหตุ', 'มูลเหตุสันนิษฐาน',
             'ลักษณะการเกิดอุบัติเ', 'สภาพอากาศ']

irrelevant_column = ['ปีที่เกิดเหตุ', 'วันที่เกิดเหตุ', 'เวลา', 'วันที่รายงาน',
                     'เวลาที่รายงาน', 'ACC_CODE', 'หน่วยงาน', 'สายทางหน่วยงาน', 
                     'รหัสสายทาง','สายทาง', 'KM']

model_2 = cleaned_2_df.drop(to_delete,axis=1)
model_2 = model_2.drop(irrelevant_column,axis=1)
model_2 = encode_categorical_features(model_2,target_col = 'มูลค่าความเสียหาย')

# Scale Different Way
model_2_standard = scale_data(model_2,scale_by='standard_scaler')
model_2_minmax = scale_data(model_2,scale_by='min_max')

NameError: name 'fill_province_and_add_region' is not defined

In [47]:
model_2_minmax.describe()

,จำนวนรถที่เกิดเหตุ,จำนวนที่เกิดเหตุทั้ง,รถจักรยานยนต์,รถสามล้อเครื่อง,รถยนต์นั่งส่วนบุคคล,รถตู้,รถปิคอัพโดยสาร,รถโดยสารมากกว่า4ล้อ,รถปิคอัพบรรทุก4ล้อ,รถบรรทุก6ล้อ,รถบรรทุกไม่เกิน10ล้อ,รถบรรทุกมากกว่า10ล้อ,รถอีแต๋น,อื่นๆ,คนเดินเท้า,จำนวนผู้เสียชีวิต,จำนวนผู้บาดเจ็บสาหัส,จำนวนผู้บาดเจ็บเล็ก,รวมจำนวนผู้บาดเจ็บ,กลุ่มบริเวณที่เกิดเหตุ_ทางตรง,กลุ่มบริเวณที่เกิดเหตุ_ทางเชื่อม,กลุ่มบริเวณที่เกิดเหตุ_ทางแยก,กลุ่มบริเวณที่เกิดเหตุ_ทางโค้ง,กลุ่มบริเวณที่เกิดเหตุ_อื่น ๆ,กลุ่มลักษณะการเกิดเหตุ_ชนกับคนหรือสิ่งของ,กลุ่มลักษณะการเกิดเหตุ_ชนกับยานพาหนะ,กลุ่มลักษณะการเกิดเหตุ_พลิกคว่ำ/ตกถนน,กลุ่มลักษณะการเกิดเหตุ_อื่นๆ,กลุ่มมูลเหตุ_การไม่เคารพกฎจราจร,กลุ่มมูลเหตุ_ความไม่พร้อมของผู้ขับขี่,กลุ่มมูลเหตุ_ถนนและสิ่งแวดล้อม,กลุ่มมูลเหตุ_พฤติกรรมผู้ขับขี่,กลุ่มมูลเหตุ_สิ่งตัดหน้า/คาดไม่ถึง,กลุ่มมูลเหตุ_สิ่งรบกวน/สมาธิ,กลุ่มมูลเหตุ_อื่นๆ,กลุ่มมูลเหตุ_อุปกรณ์/ระบบยานพาหนะ,กลุ่มมูลเหตุ_ไม่ชำนาญ/ไม่รู้เส้นทาง,กลุ่มสภาพอากาศ_ภัยธรรมชาติ,กลุ่มสภาพอากาศ_วิสัยทัศน์ไม่ดี,กลุ่มสภาพอากาศ_อากาศแจ่มใส,กลุ่มสภาพอากาศ_อื่นๆ,vehicle_group_คนเดินเท้า,vehicle_group_รถขนาดเล็ก/ส่วนบุคคล,vehicle_group_รถบรรทุก/ขนส่งสินค้า,vehicle_group_รถโดยสารสาธารณะ,vehicle_group_อื่นๆ,ภาค_กรุงเทพ,ภาค_กลาง,ภาค_ตะวันตก,ภาค_ตะวันออก,ภาค_ตะวันออกเฉียงเหนือ,ภาค_เหนือ,ภาค_ใต้,มูลค่าความเสียหาย
count,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.0,19391.000000,19391.00000,19391.000000,19391.000000,19391.0,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.00000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000,19391.000000
mean,0.090099,0.054790,0.054368,0.003223,0.163818,0.014053,0.002759,0.0,0.208169,0.03169,0.015815,0.024259,0.0,0.010182,0.001628,0.005262,0.014147,0.236171,0.276613,0.821412,0.011552,0.010623,0.151771,0.004641,0.064360,0.359548,0.55371,0.022382,0.014027,0.092466,0.030272,0.729307,0.077459,0.002424,0.007220,0.039090,0.007736,0.000309,0.115260,0.883812,0.000619,0.001805,0.787272,0.076118,0.006292,0.128513,0.128823,0.240524,0.044402,0.134495,0.180754,0.136507,0.134495,501511.062142
std,0.046961,0.042405,0.116018,0.040337,0.248379,0.084948,0.038070,0.0,0.244918,0.12609,0.073796,0.071277,0.0,0.038945,0.018986,0.017993,0.054111,0.313436,0.325092,0.383017,0.106859,0.102524,0.358808,0.067971,0.245399,0.479880,0.49712,0.147925,0.117606,0.289690,0.171339,0.444329,0.267325,0.049174,0.084665,0.193815,0.087613,0.017588,0.319343,0.320458,0.024870,0.042448,0.409247,0.265193,0.079072,0.334669,0.335012,0.427413,0.205992,0.341193,0.384824,0.343335,0.341193,288877.896327
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,116.000000
25%,0.058824,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,250751.500000
50%,0.058824,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.00

In [46]:
model_2_standard

,จำนวนรถที่เกิดเหตุ,จำนวนที่เกิดเหตุทั้ง,รถจักรยานยนต์,รถสามล้อเครื่อง,รถยนต์นั่งส่วนบุคคล,รถตู้,รถปิคอัพโดยสาร,รถโดยสารมากกว่า4ล้อ,รถปิคอัพบรรทุก4ล้อ,รถบรรทุก6ล้อ,รถบรรทุกไม่เกิน10ล้อ,รถบรรทุกมากกว่า10ล้อ,รถอีแต๋น,อื่นๆ,คนเดินเท้า,จำนวนผู้เสียชีวิต,จำนวนผู้บาดเจ็บสาหัส,จำนวนผู้บาดเจ็บเล็ก,รวมจำนวนผู้บาดเจ็บ,กลุ่มบริเวณที่เกิดเหตุ_ทางตรง,กลุ่มบริเวณที่เกิดเหตุ_ทางเชื่อม,กลุ่มบริเวณที่เกิดเหตุ_ทางแยก,กลุ่มบริเวณที่เกิดเหตุ_ทางโค้ง,กลุ่มบริเวณที่เกิดเหตุ_อื่น ๆ,กลุ่มลักษณะการเกิดเหตุ_ชนกับคนหรือสิ่งของ,กลุ่มลักษณะการเกิดเหตุ_ชนกับยานพาหนะ,กลุ่มลักษณะการเกิดเหตุ_พลิกคว่ำ/ตกถนน,กลุ่มลักษณะการเกิดเหตุ_อื่นๆ,กลุ่มมูลเหตุ_การไม่เคารพกฎจราจร,กลุ่มมูลเหตุ_ความไม่พร้อมของผู้ขับขี่,กลุ่มมูลเหตุ_ถนนและสิ่งแวดล้อม,กลุ่มมูลเหตุ_พฤติกรรมผู้ขับขี่,กลุ่มมูลเหตุ_สิ่งตัดหน้า/คาดไม่ถึง,กลุ่มมูลเหตุ_สิ่งรบกวน/สมาธิ,กลุ่มมูลเหตุ_อื่นๆ,กลุ่มมูลเหตุ_อุปกรณ์/ระบบยานพาหนะ,กลุ่มมูลเหตุ_ไม่ชำนาญ/ไม่รู้เส้นทาง,กลุ่มสภาพอากาศ_ภัยธรรมชาติ,กลุ่มสภาพอากาศ_วิสัยทัศน์ไม่ดี,กลุ่มสภาพอากาศ_อากาศแจ่มใส,กลุ่มสภาพอากาศ_อื่นๆ,vehicle_group_คนเดินเท้า,vehicle_group_รถขนาดเล็ก/ส่วนบุคคล,vehicle_group_รถบรรทุก/ขนส่งสินค้า,vehicle_group_รถโดยสารสาธารณะ,vehicle_group_อื่นๆ,ภาค_กรุงเทพ,ภาค_กลาง,ภาค_ตะวันตก,ภาค_ตะวันออก,ภาค_ตะวันออกเฉียงเหนือ,ภาค_เหนือ,ภาค_ใต้,มูลค่าความเสียหาย
0,-0.666008,-0.266765,1.686257,-0.079908,-0.659565,-0.165434,-0.072474,0.0,-0.849975,-0.251334,-0.214312,-0.340353,0.0,-0.261458,-0.085758,-0.292471,-0.261461,0.522700,0.379555,0.466279,-0.108105,-0.103622,-0.422998,-0.068286,-0.262272,-0.749265,0.897774,-0.151307,-0.119276,3.132864,-0.176683,-1.641410,-0.289762,-0.049292,-0.085278,-0.201694,-0.088294,-0.017593,-0.360936,0.362577,-0.024884,-0.042523,0.519816,-0.287035,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,-0.469717,-0.397601,2.536769,62536
1,-0.666008,-0.266765,1.686257,-0.079908,-0.659565,-0.165434,-0.072474,0.0,-0.849975,-0.251334,-0.214312,-0.340353,0.0,-0.261458,-0.085758,-0.292471,-0.261461,0.522700,0.379555,0.466279,-0.108105,-0.103622,-0.422998,-0.068286,-0.262272,-0.749265,0.897774,-0.151307,-0.119276,3.132864,-0.176683,-1.641410,-0.289762,-0.049292,-0.085278,-0.201694,-0.088294,-0.017593,-0.360936,0.362577,-0.024884,-0.042523,0.519816,-0.287035,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,-0.469717,2.515085,-0.394202,653759
2,-0.666008,-0.266765,1.686257,-0.079908,-0.659565,-0.165434,-0.072474,0.0,-0.849975,-0.251334,-0.214312,-0.340353,0.0,-0.261458,-0.085758,-0.292471,-0.261461,0.522700,0.379555,0.466279,-0.108105,-0.103622,-0.422998,-0.068286,-0.262272,-0.749265,0.897774,-0.151307,-0.119276,3.132864,-0.176683,-1.641410,-0.289762,-0.049292,-0.085278,-0.201694,-0.088294,-0.017593,-0.360936,0.362577,-0.024884,-0.042523,0.519816,-0.287035,-0.07957,-0.384011,-0.384541,1.776961,-0.215558,-0.394202,-0.469717,-0.397601,-0.394202,423925
3,-0.666008,1.271244,1.686257,-0.079908,-0.659565,-0.165434,-0.072474,0.0,-0.849975,-0.251334,-0.214312,-0.340353,0.0,-0.261458,14.963383,-0.292471,1.791997,0.522700,1.610010,0.466279,-0.108105,-0.103622,-0.422998,-0.068286,3.812828,-0.749265,-1.113866,-0.151307,-0.119276,-0.319197,-0.176683,0.609232,-0.289762,-0.049292,-0.085278,-0.201694,-0.088294,-0.017593,-0.360936,0.362577,-0.024884,-0.042523,0.519816,-0.287035,-0.07957,-0.384011,-0.384541,-0.562759,4.639126,-0.394202,-0.469717,-0.397601,-0.394202,406010
4,-0.666008,-0.266765,-0.468627,-0.079908,-0.659565,-0.165434,-0.072474,0.0,0.783267,-0.251334,-0.214312,-0.340353,0.0,-0.261458,-0.085758,-0.292471,-0.261461,0.522700,0.379555,0.466279,-0.108105,-0.103622,-0.422998,-0.068286,-0.262272,-0.749265,0.897774,-0.151307,-0.119276,3.132864,-0.176683,-1.641410,-0.289762,-0.049292,-0.085278,-0.201694,-0.088294,-0.017593,-0.360936,0.362577,-0.024884,-0.042523,0.519816,-0.287035,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,2.128939,-0.397601,-0.394202,454972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [45]:
model_2

,จำนวนรถที่เกิดเหตุ,จำนวนที่เกิดเหตุทั้ง,รถจักรยานยนต์,รถสามล้อเครื่อง,รถยนต์นั่งส่วนบุคคล,รถตู้,รถปิคอัพโดยสาร,รถโดยสารมากกว่า4ล้อ,รถปิคอัพบรรทุก4ล้อ,รถบรรทุก6ล้อ,รถบรรทุกไม่เกิน10ล้อ,รถบรรทุกมากกว่า10ล้อ,รถอีแต๋น,อื่นๆ,คนเดินเท้า,จำนวนผู้เสียชีวิต,จำนวนผู้บาดเจ็บสาหัส,จำนวนผู้บาดเจ็บเล็ก,รวมจำนวนผู้บาดเจ็บ,กลุ่มบริเวณที่เกิดเหตุ_ทางตรง,กลุ่มบริเวณที่เกิดเหตุ_ทางเชื่อม,กลุ่มบริเวณที่เกิดเหตุ_ทางแยก,กลุ่มบริเวณที่เกิดเหตุ_ทางโค้ง,กลุ่มบริเวณที่เกิดเหตุ_อื่น ๆ,กลุ่มลักษณะการเกิดเหตุ_ชนกับคนหรือสิ่งของ,กลุ่มลักษณะการเกิดเหตุ_ชนกับยานพาหนะ,กลุ่มลักษณะการเกิดเหตุ_พลิกคว่ำ/ตกถนน,กลุ่มลักษณะการเกิดเหตุ_อื่นๆ,กลุ่มมูลเหตุ_การไม่เคารพกฎจราจร,กลุ่มมูลเหตุ_ความไม่พร้อมของผู้ขับขี่,กลุ่มมูลเหตุ_ถนนและสิ่งแวดล้อม,กลุ่มมูลเหตุ_พฤติกรรมผู้ขับขี่,กลุ่มมูลเหตุ_สิ่งตัดหน้า/คาดไม่ถึง,กลุ่มมูลเหตุ_สิ่งรบกวน/สมาธิ,กลุ่มมูลเหตุ_อื่นๆ,กลุ่มมูลเหตุ_อุปกรณ์/ระบบยานพาหนะ,กลุ่มมูลเหตุ_ไม่ชำนาญ/ไม่รู้เส้นทาง,กลุ่มสภาพอากาศ_ภัยธรรมชาติ,กลุ่มสภาพอากาศ_วิสัยทัศน์ไม่ดี,กลุ่มสภาพอากาศ_อากาศแจ่มใส,กลุ่มสภาพอากาศ_อื่นๆ,vehicle_group_คนเดินเท้า,vehicle_group_รถขนาดเล็ก/ส่วนบุคคล,vehicle_group_รถบรรทุก/ขนส่งสินค้า,vehicle_group_รถโดยสารสาธารณะ,vehicle_group_อื่นๆ,ภาค_กรุงเทพ,ภาค_กลาง,ภาค_ตะวันตก,ภาค_ตะวันออก,ภาค_ตะวันออกเฉียงเหนือ,ภาค_เหนือ,ภาค_ใต้,มูลค่าความเสียหาย
0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,62536
1,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,653759
2,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,423925
3,1.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,406010
4,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,454972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19386,2.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,857762
19387,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,141167
19388,4.0,10.0,0.0,0.0,1.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,2.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,764155
19389,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,657271


In [39]:
cleaned_2_df.to_csv('./output/data_after_clean_2.csv',encoding='utf-8-sig',index = False)

# Model Development

In [35]:
model_2_minmax[numerical_feature]

,รถจักรยานยนต์,รถสามล้อเครื่อง,รถยนต์นั่งส่วนบุคคล,รถตู้,รถปิคอัพโดยสาร,รถโดยสารมากกว่า4ล้อ,รถปิคอัพบรรทุก4ล้อ,รถบรรทุก6ล้อ,รถบรรทุกไม่เกิน10ล้อ,รถบรรทุกมากกว่า10ล้อ,รถอีแต๋น,อื่นๆ,คนเดินเท้า,จำนวนผู้เสียชีวิต,จำนวนผู้บาดเจ็บสาหัส,จำนวนผู้บาดเจ็บเล็ก,รวมจำนวนผู้บาดเจ็บ
0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.4,0.4
1,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.4,0.4
2,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.4,0.4
3,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.111111,0.4,0.8
4,0.00,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.4,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19386,0.25,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.4,0.4
19387,0.00,0.0,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
19388,0.00,0.0,0.4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,1.0
19389,0.00,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.4,0.4


In [36]:
model_2_standard

,ปีที่เกิดเหตุ,ACC_CODE,KM,จำนวนรถที่เกิดเหตุ,จำนวนที่เกิดเหตุทั้ง,รถจักรยานยนต์,รถสามล้อเครื่อง,รถยนต์นั่งส่วนบุคคล,รถตู้,รถปิคอัพโดยสาร,...,vehicle_group_รถโดยสารสาธารณะ,vehicle_group_อื่นๆ,ภาค_กรุงเทพ,ภาค_กลาง,ภาค_ตะวันตก,ภาค_ตะวันออก,ภาค_ตะวันออกเฉียงเหนือ,ภาค_เหนือ,ภาค_ใต้,มูลค่าความเสียหาย
0,0.0,-0.020822,-0.534353,-0.666008,-0.266765,1.686257,-0.079908,-0.659565,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,-0.469717,-0.397601,2.536769,62536
1,0.0,-2.854984,0.209219,-0.666008,-0.266765,1.686257,-0.079908,-0.659565,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,-0.469717,2.515085,-0.394202,653759
2,0.0,-2.853407,-0.355515,-0.666008,-0.266765,1.686257,-0.079908,-0.659565,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,1.776961,-0.215558,-0.394202,-0.469717,-0.397601,-0.394202,423925
3,0.0,-2.853261,-0.552340,-0.666008,1.271244,1.686257,-0.079908,-0.659565,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,-0.562759,4.639126,-0.394202,-0.469717,-0.397601,-0.394202,406010
4,0.0,-2.853176,-0.263526,-0.666008,-0.266765,-0.468627,-0.079908,-0.659565,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,2.128939,-0.397601,-0.394202,454972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19386,0.0,0.808274,1.423492,0.586617,0.245905,1.686257,-0.079908,0.950915,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,-0.562759,4.639126,-0.394202,-0.469717,-0.397601,-0.394202,857762
19387,0.0,0.802769,0.176890,0.586617,-0.266765,-0.468627,-0.079908,0.950915,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,2.128939,-0.397601,-0.394202,141167
19388,0.0,0.784017,-0.306386,3.091867,3.834591,-0.468627,-0.079908,0.950915,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,2.128939,-0.397601,-0.394202,764155
19389,0.0,0.788947,-0.473327,-0.666008,-0.266765,-0.468627,-0.079908,0.950915,-0.165434,-0.072474,...,-0.07957,-0.384011,-0.384541,-0.562759,-0.215558,-0.394202,2.128939,-0.397601,-0.394202,657271


# ทด

In [12]:
def yeo_johnson_transform(df, columns):
    df = df.copy()  # ป้องกันการเปลี่ยนแปลง DataFrame ต้นฉบับ
    pt = PowerTransformer(method='yeo-johnson')  # ใช้วิธี Yeo-Johnson

    for column in columns:
        if column in df.columns and pd.api.types.is_numeric_dtype(df[column]):
            non_na_values = df[column].dropna().values.reshape(-1, 1)  # เอาค่าที่ไม่ใช่ NaN
            if len(np.unique(non_na_values)) > 1:  # เช็คว่ามีค่าหลายค่า (ไม่นับ NaN)
                transformed_values = pt.fit_transform(non_na_values).flatten()
                df.loc[df[column].notna(), column] = transformed_values
    return df

In [ ]:
def log_transform(df, columns):
    df = df.copy()  # ป้องกันการเปลี่ยนแปลง DataFrame ต้นฉบับ
    for column in columns:
        if column in df.columns and pd.api.types.is_numeric_dtype(df[column]):
            df[column] = np.log(df[column])  # ใช้ log(1+x) เพื่อจัดการค่า 0
        else:
            print(f"Column '{column}' ไม่ใช่ตัวเลขหรือไม่มีอยู่ใน DataFrame")
    return df